In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import f1_score, make_scorer, mean_squared_error
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from collections import Counter
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests 

In [2]:
#r = requests.get(url = "http://www.omdbapi.com/?i=tt3896198&apikey=2d492c2e")
r = requests.get(url='http://www.omdbapi.com/?apikey=2d492c2e&i=tt3896198')
#print(r.json
y = r.json()
print(y["Ratings"])
for x in y["Ratings"]:
    print(x.get('Value'))
    print(type(x.get('Value')))
#print(type(r.json()))
#print(r.json().get("Ratings"))

[{'Source': 'Internet Movie Database', 'Value': '7.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '85%'}, {'Source': 'Metacritic', 'Value': '67/100'}]
7.6/10
<class 'str'>
85%
<class 'str'>
67/100
<class 'str'>


In [11]:
#df = pd.read_csv("train.csv", header=0) 
df.belongs_to_collection.unique()[1]

"[{'id': 107674, 'name': 'The Princess Diaries Collection', 'poster_path': '/wt5AMbxPTS4Kfjx7Fgm149qPfZl.jpg', 'backdrop_path': '/zSEtYD77pKRJlUPx34BJgUG9v1c.jpg'}]"

In [28]:
print(df.describe())
print("\n")
print((list(df.columns)))
print("\n")
print(df.production_companies)
print(df.dtypes)

#### Look into plotting revenue by production agencies.
#Get the discorete agencies from each production, put that into a dictionary with the revenue from that movie.
#Then we can just plot that revenue against the agencies. 

                id        budget   popularity      runtime       revenue
count  3000.000000  3.000000e+03  3000.000000  2998.000000  3.000000e+03
mean   1500.500000  2.253133e+07     8.463274   107.856571  6.672585e+07
std     866.169729  3.702609e+07    12.104000    22.086434  1.375323e+08
min       1.000000  0.000000e+00     0.000001     0.000000  1.000000e+00
25%     750.750000  0.000000e+00     4.018053    94.000000  2.379808e+06
50%    1500.500000  8.000000e+06     7.374861   104.000000  1.680707e+07
75%    2250.250000  2.900000e+07    10.890983   118.000000  6.891920e+07
max    3000.000000  3.800000e+08   294.337037   338.000000  1.519558e+09


['id', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue']


0       [{'name': 'P

In [6]:
train_data = pd.read_csv("train.csv", usecols=["budget", "popularity", "runtime", "revenue"]) 
test_data = pd.read_csv("test.csv", usecols=["budget", "popularity", "runtime"]) 
test_data = test_data.dropna()
train_data = train_data.dropna()
Y_train = train_data["revenue"]

X_train = train_data.drop("revenue", axis=1)

X_test = test_data

X_train = X_train.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
X_test = X_test.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [31]:
df.production_companies[0]
l = [df.production_companies[0]]
print(l)

["[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'United Artists', 'id': 60}, {'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8411}]"]


In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
sc_transform = scaler.transform(X_train)
X_train = pd.DataFrame(sc_transform)

In [ ]:
X_train.head()

In [ ]:
svc = SVC()
svc.fit(X_train, Y_train)
model_pred = svc.predict(X_train)
model_f1 = f1_score(Y_train, model_pred)

print('Support Vector Classifier F1 score: %f' % model_f1)

In [ ]:
svc = SVC()
svc.fit(X_train, Y_train)
model_pred = svc.predict(X_train)

svc_scores = cross_val_score(svc, X_train, Y_train, cv=5, scoring = make_scorer(f1_score))
print(svc_scores)
print('cv_scores mean:{}'.format(np.mean(svc_scores)))

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, Y_train)
model_pred = dt.predict(X_train)

dt_scores = cross_val_score(dt, X_train, Y_train, cv=5, scoring = make_scorer(f1_score))
print(dt_scores)
print('cv_scores mean:{}'.format(np.mean(dt_scores)))

In [12]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error
lr = LogisticRegression()
lr.fit(X_train, Y_train)

model_pred = lr.predict(X_train)
#y = mean_squared_error(Y_train, model_pred,squared=False)
lr_scores = cross_val_score(lr, X_train, Y_train, cv=5, scoring = make_scorer(mean_squared_error(Y_train, model_pred,squared=False)))
print(lr_scores)
print('cv_scores mean:{}'.format(np.mean(lr_scores)))

/Users/amangangwani/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AttributeError: 'numpy.float64' object has no attribute '__name__'

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
model_pred = knn.predict(X_train)

knn_scores = cross_val_score(knn, X_train, Y_train, cv=5, scoring = make_scorer(f1_score))
print(knn_scores)
print('cv_scores mean:{}'.format(np.mean(knn_scores)))

In [ ]:
nb = GaussianNB()
nb.fit(X_train, Y_train)
model_pred = nb.predict(X_train)

nb_scores = cross_val_score(nb, X_train, Y_train, cv=5, scoring = make_scorer(f1_score))
print(nb_scores)
print('cv_scores mean:{}'.format(np.mean(nb_scores)))

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
neighbors = list(range(1,30))

hyperparameters = dict(n_neighbors=neighbors)

In [ ]:
knn_2 = KNeighborsClassifier(p=2)

clf = GridSearchCV(knn_2, hyperparameters, cv=5)

clf.fit(X_train, Y_train)
clf.grid_scores_

In [ ]:
clf_mean_scores = [result.mean_validation_score for result in clf.grid_scores_]

In [ ]:
plt.plot(neighbors, clf_mean_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')

In [ ]:
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

In [ ]:
neighbors_val = best_model.best_estimator_.get_params()['n_neighbors']

In [ ]:
model = KNeighborsClassifier(n_neighbors = neighbors_val)
model.fit(X_train, Y_train)
model_pred = model.predict(X_train)
model_f1 = f1_score(Y_train, model_pred)

print('KNN Model F1 score after hyperparameter tuning: %f' % model_f1)

In [ ]:
Y_pred = model.predict(X_test)

test_data["class"] = Y_pred
result = test_data[["ID","class"]]
result.to_csv("result.csv", index=False)
result.head()

In [ ]:
result.describe()